# 3. Properties of gas mixtures

## 3.1. Binary CO$_2$-CH$_4$ mixture

In [ ]:
import numpy as np
import xarray as xr

from dartsflash.libflash import FlashParams, EoSParams, EoS, InitialGuess
from dartsflash.libflash import Flash
from dartsflash.libflash import CubicEoS

from dartsflash.dartsflash import DARTSFlash, CompData
from dartsflash.mixtures import VL

components = ["CO2", "C1"]
comp_data = CompData(components, setprops=True)
f = VL(comp_data=comp_data)

f.set_vl_eos("PR", trial_comps=[0, 1], 
             root_order=[EoS.MAX, EoS.MIN],
             # rich_phase_order=[0, -1],  # will sort rich liquid phases according to component index. In this case: [CO2-rich liquid, other liquid]
            )
ceos = f.eos["VL"]

f.init_flash(flash_type=DARTSFlash.FlashType.PTFlash)

## 3.2. PT-diagram

In [ ]:
""" Define state specifications and compositions """
dz = 0.01
min_z = [0.9]
max_z = [1.]
compositions = {"CO2": np.arange(min_z[0], max_z[0]+dz*0.1, dz),
                "C1": 1.,
               }
state_spec = {"pressure": np.arange(40, 90, 1),
              "temperature": np.arange(270, 300, 1),
              }

""" Perform flash calculations """
pt_flash_results = f.evaluate_flash(state_spec=state_spec, compositions=compositions, mole_fractions=True, print_state="FLASH")
props_tot = f.evaluate_properties_np(state_spec=state_spec, compositions=compositions,
                                     state_variables=f.state_vars + components, flash_results=pt_flash_results,
                                     total_properties_to_evaluate={"H": EoS.Property.ENTHALPY})

# Plot flash results
from dartsflash.plot import PlotFlash
pt = PlotFlash.pt(f, pt_flash_results, composition=[0.9, 0.1], plot_phase_fractions=False, pt_props=props_tot,
                 # critical_point={"pressure": 79, "temperature": 296.5}
                 )


### 3.3. PH diagram of binary mixture

In [ ]:
#  PH diagram
from dartsflash.dartsflash import DARTSFlash, CompData
from dartsflash.mixtures import VL
comp_data = CompData(components, setprops=True)
f = VL(comp_data=comp_data)

f.set_vl_eos("PR")
f.init_flash(flash_type=DARTSFlash.FlashType.PHFlash,
             t_min=100., t_max=500., t_init=300.,  # temperature bracketing
             split_switch_tol=1e-1
            )

# Evaluate PH-flash
Prange = [10., 100.]
Trange = [260., 350.]
z = [0.9, 0.1]
Hrange = f.get_ranges(prange=Prange, trange=Trange, composition=z)
state_spec = {"pressure": np.linspace(Prange[0], Prange[1], 100),
              "enthalpy": np.linspace(Hrange[0], Hrange[1], 100)
             }
compositions = {"CO2": z[0], "C1": z[1]}

ph_flash_results = f.evaluate_flash(state_spec=state_spec, compositions=compositions, mole_fractions=True, print_state="PH flash")

# Plot PH diagram
from dartsflash.plot import PlotFlash
ph = PlotFlash.ph(f, ph_flash_results, composition=z, min_temp=250., max_temp=350., min_val=0., max_val=1.,
                  plot_phase_fractions=True, pt_props=None)
